In [1]:
#!/usr/bin/env python3.4
# Python variables are local, if not otherwise declared

'''
This script compares deep-seq reads of transcripts - updated 06/03/16.\n
This script identifies all the transcripts with at least a given number of reads in either the WT \
or the mutant, and return a list of transcripts with a ratio of mut/(wt+mut) below a given value in a file \
named as mutant_wt_type_strand_small-RNA-cutoff_normalization-standard_depletion-threshold.txt\n
Copy this script into a working folder with all the sample_named subfolders each containing
two files: sum_sample.txt and sum_sample_17_cdna_type_normed.txt\n
Four files are also required in the working folder containing this script
wago_targets_at_least_three_overlap.txt
csr1_targets_at_least_three_overlap.txt
ERGO-1_targets.txt
alg34_fog2_20_twice.txt\n
There are 6 normalization methods:
1) nons; 2) cose,coan,coto; 3) mir; 4) tot\n
'''

import os
import operator
import re
import sys

tp = 'cdna_coding' # cdna_coding, cdna_miRNA, cdna_u21RNA, ......
stan = 'cose' # normalization standard, using tot, nons, cose (coding sen), coan (coding anti),coto (coding tot), mir, or 21U
strand ='se'; # se for sense, an for anti, and all for both sense and anti
cut=1; # read cutoff / gene
thres=2; # depletion ratio (0-1): MT/(MT+WT) if more than 1, that means all

# wago, csr, alg, ergo, other = 'blue','red','cyan','black','green'

min_size = input('size analyzed using 17 for all, 22 for 22G, and 26 for 26mer:')

sam1 = input('WT:')
sam11 = input('Normalization sample for WT:')
sam2 = input('Mutant:')
sam22 = input('Normalization sample for Mutant:')

### name the output directory
names = [sam1, sam2]
names = sorted(names)
name = names[0]+'_'+names[1]

def main():
    ### normalization
    wt = norm_data(sam1, sam11)
    mut = norm_data(sam2, sam22)

    #if not os.path.isdir(name):
    #    os.mkdir(name)
  
    ### make a directory, if there already exists a file with the directory name, raise Error  
    try:
        os.makedirs('mRNA_comparison/'+name)
    except OSError:
        if not os.path.isdir('mRNA_comparison/'+name):
            raise
        
    ### ????? cut off 1, remove genes which have less than 1 read
    wt_keys = [k for k in wt.keys()]
    for ks in wt_keys:
        if wt[ks] < cut and mut[ks] < cut:
            del wt[ks]
            del mut[ks]
            
        
    ### how many genes have at least 1 read in either mut and/or wt
    count = len(wt)
    for ks in mut.keys():
        if ks not in wt.keys():
            count += 1
    print (count, 'genes with at least', cut, 'small RNAs in either', sam1, 'and/or', sam2, '\n')


    depe = {}
    for ks in mut.keys():
        depe[ks] = mut[ks]/(wt[ks]+mut[ks])

    # sorted(depe.items(), key=lambda depe: depe[1])
    sorted_depe = sorted(depe.items(), key = operator.itemgetter(1))

    f = open('mRNA_comparison/'+name+'/'+sam2+'_'+sam1+'_'+tp+'_'+strand+'_'+str(cut)+'_'+stan+'_'+str(thres)+'.txt', 'wt')

    #orig_stdout = sys.stdout
    #sys.stdout = f

    for (key, val) in sorted_depe: # *, 0 or more; +, 1 or more; ?, 0 or 1
        if not val <= thres:
            continue
        m = str(round(mut[key],4))
        w = str(round(wt[key],4))
        r = str(round(depe[key],4))
    
        f.write(key+'\t'+m+'\t'+w+'\t'+r+'\n') 
        #print (ks+'\t'+m+'\t'+w+'\t'+r+'\n')
    
    #sys.stdout = orig_stdout
    f.close()

    #sorted_depe.clear() # also clear depe
    del sorted_depe[:]

    file = sam2+'_'+sam1+'_'+tp+'_'+strand+'_'+str(cut)+'_'+stan+'_'+str(thres)+'.txt'
    label_gene(file)

    
def norm(sam, stan):
    # get the normalization ratio
    geex, struc, cose, coan, coto, mir, u21 = 0, 0, 0, 0, 0, 0, 0
    file = sam+'/sum_'+sam+'.txt'
    with open (file) as f:
        for line in f:
            #line = re.sub(r'\s+$', r'', line)
            line = line.rstrip()
            if re.search(r'^geex\t\S+\t\S+\t\S+', line):
                mch = re.search(r'^geex\t\S+\t\S+\t(\S+)', line) # all
                geex = float(mch.group(1))
            if (re.search(r'^struc\t\S+\t\S+\t\S+', line)):
                mch = re.search(r'^struc\t(\S+)\t\S+\t\S+', line) # sense
                struc = float(mch.group(1))
            if (re.search(r'^coding\t\S+\t\S+\t\S+', line)):
                mch = re.search(r'^coding\t(\S+)\t(\S+)\t(\S+)', line)
                cose = float(mch.group(1)); coan = float(mch.group(2)); coto = float(mch.group(3))
            if (re.search(r'^miRNA\t\S+\t\S+\t\S+', line)):
                mch = re.search(r'miRNA\t(\S+)\t\S+\t\S+', line) # sense
                mir = float(mch.group(1))
            if (re.search(r'^u21RNA\t\S+\t\S+\t\S+', line)):
                mch = re.search(r'u21RNA\t(\S+)\t\S+\t\S+', line) # sense
                u21 = float(mch.group(1))
            if (re.search(r'^star-miRNA', line)):
                break
    nons = geex - struc
        
    print ('''total %.2f
              sense struc %.2f
              non-struc %.2f
              sense coding %.2f
              anti coding %.2f
              total coding %.2f
              sense miRNA %.2f
              sense 21U-RNA %.2f\n''' % (geex, struc, nons, cose, coan, coto, mir, u21))
        
    if (stan == 'mir'):
        ratio = 1000000/mir
    elif (stan == 'cose'):
        ratio = 5000000/cose
    elif (stan == 'coan'):
        ratio = 2000000/coan
    elif (stan == 'coto'):
        ratio = 5000000/coto
    elif (stan == 'nons'):
        ratio = 5000000/nons
    elif (stan == 'tot'):
        ratio = 5000000/geex
    elif (stan == '21U'):
        ratio = 500000/u21
    
    print ('Normalization ratio', ratio, '\n\n')
    
    return ratio


def norm_data(sam, nsam):
    # returns a dictionary
    # 'sum_0320tocsr1om_all_all_csr1_3utr_17_cdna_coding_mir.txt' what's this file? not normalized ??? change ratio to 1 ?
    # 'sum_0320tocsr1om_all_17_cdna_coding.txt' how about this ?
    al = {}
    ra = norm(nsam, stan)
    print ('using', nsam, ',', sam, 'normaization ratio is', ra, '\n\n')
    file = sam+'/sum_'+sam+'_'+min_size+'_'+tp+'.txt'
    with open(file) as f:
        for line in f:
            #line = re.sub(r'\s+$', r'', line)
            line = line.rstrip()
            if line.startswith('>'):
                continue
            lst = line.split('\t')
            gene = lst[0]+'\t'+lst[1]
            if (strand == 'se'):
                al[gene] = float(lst[2])*ra
            elif (strand == 'an'):
                al[gene] = float(lst[3])*ra
            elif (strand == 'all'):
                al[gene] = float(lst[4])*ra
    count = len(al)

    print (sam, tp, 'total genes:', count, '\n\n')
    
    return al


def label_gene(file):
    wago = {}; csr = {}; ergo = {}; alg = {}; emb = {}
    with open('wago_targets_at_least_three_overlap.txt', 'rt') as f:
        for line in f:
            lst = line.rstrip().split('\t')
            wago[lst[0]]=1
    
    with open('csr1_targets_at_least_three_overlap.txt', 'rt') as f:
        for line in f:
            lst = line.rstrip().split('\t')
            csr[lst[0]]=1
    
    with open('ERGO-1_targets.txt', 'rt') as f:
        for line in f:
            lst = line.rstrip().split('\t')
            ergo[lst[0]]=1
    
    with open('alg34_fog2_20_twice.txt', 'rt') as f:
        for line in f:
            lst = line.rstrip().split('\t')
            alg[lst[0]]=1

    #with open('embro_targets.txt', 'rt') as f:
    #    for line in f:
    #        lst = line.rstrip().split('\t')
    #        gene = lst[0]+'\t'+lst[1]
    #        emb[gene]=1   
    

    out = re.sub(r'\.txt', r'_labeled.txt', file)
    outfile = open('mRNA_comparison/' + name + '/' + out, 'wt')
    
    infile = 'mRNA_comparison/' + name + '/' + file
    with open(infile) as f:
        for line in f:
            line = line.rstrip()
            lst = line.split('\t')
            outfile.write(line + '\t')
            if lst[0] in wago.keys():
                outfile.write('wago' + '\t')
            else:
                outfile.write('Not' + '\t')
            
            if lst[0] in csr.keys():
                outfile.write('csr' + '\t')
            else:
                outfile.write('Not' + '\t')
                
            if lst[0] in ergo.keys():
                outfile.write('ergo' + '\t')
            else:
                outfile.write('Not' + '\t')
                
            if lst[0] in alg.keys():
                outfile.write('alg' + '\n')
            else:
                outfile.write('Not' + '\n')
                
            #if lst[0] in emb.keys():
            #    outfile.write('emb' + '\n')
            #else:
            #    outfile.write('Not' + '\n')
        
    outfile.close()
    
if __name__ == '__main__':
    main()
    

size analyzed using 17 for all, 22 for 22G, and 26 for 26mer:17
WT:0320toom_all
Normalization sample for WT:0320toom_all
Mutant:0320tocsr1om_all
Normalization sample for Mutant:0320tocsr1om_all
total 6762488.00
sense struc 336050.82
non-struc 6426437.18
sense coding 100989.96
anti coding 3382585.06
total coding 3483575.02
sense miRNA 1015206.00
sense 21U-RNA 205742.87

Normalization ratio 49.50987207045136 


using 0320toom_all , 0320toom_all normaization ratio is 49.50987207045136 


0320toom_all cdna_coding total genes: 20344 


total 10310921.00
sense struc 972969.38
non-struc 9337951.62
sense coding 215111.03
anti coding 4064389.66
total coding 4279500.69
sense miRNA 1264368.17
sense 21U-RNA 1171800.61

Normalization ratio 23.24381041734587 


using 0320tocsr1om_all , 0320tocsr1om_all normaization ratio is 23.24381041734587 


0320tocsr1om_all cdna_coding total genes: 20344 


11148 genes with at least 1 small RNAs in either 0320toom_all and/or 0320tocsr1om_all 



In [4]:
### draw the figure
import os
import numpy as np
import matplotlib.pyplot as plt
from math import log2

os.getcwd()
directory = 'small_RNA_comparison/0320tocsr1om_all_0320toom_all/'

with open (directory+'0320tocsr1om_all_0320toom_all_all_3utr_cdna_coding_1_mir_2_labeled.txt', 'rt') as f:
    rdecsr = []; N2 = []
    for line in f:
        lst = line.rstrip().split()
        if lst[6] == 'csr':
            x = float(lst[2])
            y = float(lst[3])
            rdecsr.append(log2(x) if x > 1 else 0)
            N2.append(log2(y) if y > 1 else 0)
            

plt.scatter(N2, rdecsr, marker = 'o', color = 'red', s = 0.1)
plt.plot([0, 15], [0, 15], ls="-", c=".3") # logy = logx, y = x
plt.plot([0, 14], [1, 15], ls="--", c=".3") # logy = logx + 1, y = 2x
plt.plot([1, 15], [0, 14], ls="--", c=".3") # logy = logx - 1, x = 2y

plt.xlim(0,15)
plt.ylim(0,15)
plt.xlabel('0320toom (log2)')
plt.ylabel('0320tocsr1om (log2)')

plt.show()


In [6]:
import os
import numpy as np
import matplotlib.pyplot as plt
import math

os.getcwd()
directory = 'small_RNA_comparison/031616122Arde3csr1_all_031616N2ctrlforw090_all/'

with open (directory+'031616122Arde3csr1_all_031616N2ctrlforw090_all_all_3utr_cdna_coding_1_mir_2_labeled.txt', 'rt') as f:
    rdecsr = []; N2 = []
    for line in f:
        lst = line.rstrip().split()
        if lst[6] == 'csr':
            x = float(lst[2])
            y = float(lst[3])
            rde.append(x)
            N2.append(y)

            

plt.scatter(N2, rde, marker = '.', color = 'blue', s = 10)
plt.plot([0, 10], [0, 15], ls="--", c=".3")
plt.xlabel('031616N2ctrlforw090 (log2)')
plt.ylabel('031616122Arde3csr1 (log2)')
plt.xlim(0,15)
plt.ylim(0,15)
plt.show()


NameError: name 'rde' is not defined

In [7]:
math.log2(0.00000001)

-26.575424759098897

In [9]:
for x in range(0,5,1):
    print (x)
    print (log2(x) if x >= 1 else 1)

0
1
1
0.0
2
1.0
3
1.584962500721156
4
2.0


In [25]:
a = '15'
b = float(a)
print (b)
math.log2(b)

15.0


3.9068905956085187

In [32]:
mean, cov = [0, 0], [(1, .6), (.6, 1)]
x, y = np.random.multivariate_normal(mean, cov, 100).T
y += x + 1
f, ax = plt.subplots(figsize=(6, 6))
ax.scatter(x, y, c=".3")
ax.plot([-3, 3], [-3, 3], ls="--", c=".3")
ax.set(xlim=(-3, 3), ylim=(-3, 3))
plt.show()

In [11]:
x=1; y=2
print x

1


In [19]:
names = [1,2,5,3]
names = sorted(names)
print (names)

[1, 2, 3, 5]


In [23]:
import os
os.getcwd()
os.listdir()

['.DS_Store',
 '.ipynb_checkpoints',
 '0320tocsr1om_all_0320toom_all_all_3utr_cdna_coding_an_1_mir_2_labeled.txt',
 '0320tocsr1om_all_17_cdna_coding_normed.txt',
 '0320tocsr1om_all_3utr_17_cdna_coding_mir.txt',
 '7_count_utr_CDS_reads.ipynb',
 '8_transcript_comparison.ipynb',
 'alg34_fog2_20_twice.txt',
 'ceWS215_3utr_pos.txt',
 'ceWS215_CDS_3utr_filtered.txt',
 'ceWS215_CDS_5utr_filtered.txt',
 'ceWS215_CDS_pos.txt',
 'ceWS215_gdna.fa',
 'csr1_targets_at_least_three_overlap.txt',
 'ERGO-1_targets.txt',
 'folder',
 'rank_csr_targets.ipynb',
 'ranked.txt',
 'read_fastq.ipynb',
 'sum_0320tocsr1om_all.txt',
 'sum_3utr_read_CSR1_targets_mir.txt',
 'transcript_comparison_082814.pl',
 'Untitled0.ipynb',
 'wago_targets_at_least_three_overlap.txt']

In [28]:
import os
try: 
    os.mdir('ttt')
except OSError:
    if not os.path.isdir('ttt'):
        raise 

In [41]:
round(0.5975,3)

0.598

In [44]:
f = open('folder/'+ 'mmm.txt', 'wt')
f.write('x')

1

In [76]:
import sys
name = 'N2csr1'
sam1 = 'N2'
sam2 = 'csr1'
grp = 'all'
utr = '3utr'
tp = 'coding'
cut = 1
stan = 'mir'
thres = 2

f = open('folder/'+name+'/'+sam2+'_'+sam1+'_'+grp+'_'+utr+'_'+tp+'_'+str(cut)+'_'+stan+'_'+str(thres)+'.txt', 'wt')
f.write('love')
f.write('u')
f.write('forever'+'\n')
f.write('haha'+'\n')

orig_stdout = sys.stdout
sys.stdout = f

print ('Dito')
print ('Only')

sys.stdout = orig_stdout

print ('too')

too


In [53]:
import operator
x = {'a':1, 'b':5, 'e':3, 'd':2}
y = {'a':1, 'b':5, 'e':3, 'd':2}


x = sorted(x.items(), key=lambda x: x[1])
y = sorted(y.items(), key = operator.itemgetter(0))

print (x)
print (y)

[('a', 1), ('d', 2), ('e', 3), ('b', 5)]
[('a', 1), ('b', 5), ('d', 2), ('e', 3)]


In [60]:
import re
x = '1000'
pattern = r'\.?0+$'
su = r''

re.sub(pattern, su, x)

'1'

In [63]:
3.000/5

0.6

In [77]:
d = {"stuff": "things"}
d2 = d
d = {}
print (d2)

d = {"stuff": "things"}
d2 = d
d.clear()
print (d2)

d = {"stuff": "things"}
d2 = d
d2.clear()
print (d)

{'stuff': 'things'}
{}
{}


In [78]:

print ('folder/'+name+'/'+sam2+'_'+sam1+'_'+grp+'_'+utr+'_'+tp+'_'+str(cut)+'_'+stan+'_'+str(thres)+'.txt', 'wt')

file = sam2+'_'+sam1+'_'+grp+'_'+utr+'_'+tp+'_'+str(cut)+'_'+stan+'_'+str(thres)+'.txt'

print (file)

folder/N2csr1/csr1_N2_all_3utr_coding_1_mir_2.txt wt
csr1_N2_all_3utr_coding_1_mir_2.txt


In [87]:
geex, struc, cose, coan, coto, mir, u21, nons = 0,0,0,0,0,0,0,0

In [90]:
        print ('''total %.2f
sense struc %.2f
non-struc %.2f
sense coding %.2f
anti coding %.2f
total coding %.2f
sense miRNA %.2f
sense 21U-RNA %.2f\n''' % (geex, struc, nons, cose, coan, coto, mir, u21))

SyntaxError: EOL while scanning string literal (<ipython-input-90-daddf88d2c30>, line 1)

In [107]:
y = 7
import sys
name = 'N2csr1'
sam1 = 'N2'
sam2 = 'csr1'
grp = 'all'
utr = '3utr'
tp = 'coding'
cut = 1
stan = 'mir'
thres = 2

wt = {}
def test(wt):
    wt = {'a':1}
    z = 'folder/'+name+'/'+sam2+'_'+sam1+'_'+grp+'_'+utr+'_'+tp+'_'+str(cut)+'_'+stan+'_'+str(thres)+'.txt'
    #print (z)
    return wt
    
m = test(wt)

print (m)
print (wt)

{'a': 1}
{}


In [104]:

y = 7
def test(x):
    y = x + 1
    print (y)
    
test(y)
print (y)


def test(x):
    global y # parse out
    y = x + 1
    print (y)
    
test(y)
print(y)

8
7
8
8


In [113]:
print (
'''
This script compares deep-seq reads of transcripts - updated 06/03/11 by WG.\n
This script identifies all the transcripts with at least a given number of reads in either the WT \
or the mutant, and return a list of transcripts with a ratio of mut/(wt+mut) below a given value in a file \
named as mutant_wt_type_strand_small-RNA-cutoff_normalization-standard_depletion-threshold.txt\n
Copy this script into a working folder with all the sample_named subfolders each containing
two files: sum_sample.txt and sum_sample_17_cdna_type_normed.txt\n
Four files are also required in the working folder containing this script
wago_targets_at_least_three_overlap.txt
csr1_targets_at_least_three_overlap.txt
ERGO-1_targets.txt
alg34_fog2_20_twice.txt\n
There are 6 normalization methods:
1) nons; 2) cose,coan,coto; 3) mir; 4) tot\n
''')

print (x)


This script compares deep-seq reads of transcripts - updated 06/03/11 by WG.

This script identifies all the transcripts with at least a given number of reads in either the WT or the mutant, and return a list of transcripts with a ratio of mut/(wt+mut) below a given value in a file named as mutant_wt_type_strand_small-RNA-cutoff_normalization-standard_depletion-threshold.txt

Copy this script into a working folder with all the sample_named subfolders each containing
two files: sum_sample.txt and sum_sample_17_cdna_type_normed.txt

Four files are also required in the working folder containing this script
wago_targets_at_least_three_overlap.txt
csr1_targets_at_least_three_overlap.txt
ERGO-1_targets.txt
alg34_fog2_20_twice.txt

There are 6 normalization methods:
1) nons; 2) cose,coan,coto; 3) mir; 4) tot


1000


In [118]:
os.getcwd()
os.listdir()
n = 'xxx'

os.makedirs('folder/'+n)


In [119]:
try:
    os.makedirs(n)
except OSError:
    if not os.path.isdir(n):
        raise

In [126]:
def main():
    zz = 't'
    mm(zz)
    
def mm(zz):
    print (zz)
    
    
    
if __name__ == '__main__':
    main()


t


In [7]:
x, y = 1, 3
t = x
x = 7
print (t)

1


In [10]:
w = {'a':2, 'b': 3, 'c':0.5}
mu = {'a':1, 'b': 2, 'c':0.7}

w_keys = [k for k in w.keys()]

for k in w_keys:
    if w[k] < cut and mu[k] < cut:
        del w[k]
        del mu[k]
            
print (w, mu)

{'b': 3, 'a': 2} {'b': 2, 'a': 1}


In [19]:

for k, v in [(2,3),[5,7]]:
    print (k,v)

2 3
5 7


In [ ]:
plt.plot([-3, 3], [-3, 3], ls="--", c=".3")
plt.show()

In [ ]:
9 lectures
9 lectures


three experiment

